# Forward Kinematics

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import vector
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
class Segment:
    def __init__(self, x, y, l, theta, color, parent=None):
        self.a = vector.obj(x=x, y=y)
        self.l = l
        self.theta = theta
        self.parent = parent
        self.color = color
        self.calculate_b()
    
    @classmethod
    def with_parent(cls, parent, l, theta, color):
        return cls(parent.b.x, parent.b.y, l, theta,color, parent)
    
    def calculate_b(self):
        dx = self.l * round(np.cos(self.theta),2)
        dy = self.l * round(np.sin(self.theta),2)
        self.b = vector.obj(x=self.a.x+dx, y=self.a.y+dy)
    
    def update(self, theta):
        self.theta = theta
        if(self.parent != None):
            self.a = vector.obj(x=self.parent.b.x, y=self.parent.b.y)
            self.theta += self.parent.theta
        self.calculate_b()
    
    def show(self):
        x_vals = [self.a.x, self.b.x]
        y_vals = [self.a.y, self.b.y]
        plt.plot(x_vals, y_vals, f'{self.color}o', linestyle="-", linewidth=4)

In [3]:
# Number of links
N_LINKS = 2

# {length, color}
links_config = [{'l': 3, 'c': 'r'}, {'l': 2, 'c':'g'}, {'l':2, 'c':'b'}]

assert(len(links_config) >= N_LINKS)

robot = [Segment(0, 0, links_config[0]['l'], 0, links_config[0]['c'])]
for i in range(1, N_LINKS):
    robot.append(Segment.with_parent(robot[i-1], links_config[i]['l'], 0, links_config[i]['c']))

theta_sliders = [widgets.IntSlider(value=0,min=-180,max=180,step=1,description=f'Theta_{i}') for i in range(len(robot))]
theta = {f'theta_{i}':slider for i, slider in enumerate(theta_sliders)}

In [4]:
@interact(**theta)
def forward_kinematics(**kwargs):
    plt.figure(figsize=(5,5))
    plt.xlim((-6,6))
    plt.ylim((-6,6))
    for i in range(-6, 6):
        plt.plot([i,i],[-6,6], linewidth=0.1, color='black' )
        plt.plot([-6,6],[i,i], linewidth=0.1, color='black' )
    plt.plot([0,0],[-6,6], linewidth=0.5, color='black' )
    plt.plot([-6,6],[0,0], linewidth=0.5, color='black' )
    plt.title(f"{len(robot)}R Arm FK")
    for i, theta_i in enumerate(kwargs.values()):
        theta_i = theta_i * np.pi/180
        robot[i].update(theta_i)
    for link in robot:
        link.show()
    plt.legend([f"{robot[-1].b.x:.2f}, {robot[-1].b.y:.2f}"])

interactive(children=(IntSlider(value=0, description='Theta_0', max=180, min=-180), IntSlider(value=0, descrip…